In [ ]:
!pip install langchain
!pip install langchain_openai
!pip install pypdf
!pip install langchain_community
!pip install faiss-cpu
!pip install rank_bm25
!pip install langchain_experimental
!pip install rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Fou

In [ ]:
import langchain
from langchain_community.document_loaders import PyPDFLoader,Docx2txtLoader,TextLoader
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS

In [ ]:
def load_document(folderpath: str):
  documents=[]
  for filenames in os.listdir(folderpath):
    if filenames.endswith(".pdf"):
      loader=PyPDFLoader(os.path.join(folderpath,filenames))
    elif filenames.endswith(".docx"):
      loader=Docx2txtLoader(os.path.join(folderpath,filenames))
    else:
      raise ValueError("File type not supported")
      continue
    documents.extend(loader.load())
  return documents

In [ ]:
def generate_response(prompt):
  prompt_response=ChatPromptTemplate.from_template("{topic}")
  response=prompt_response.invoke({"topic":prompt})
  chat_model=ChatOpenAI()
  response=chat_model(response)
  return response

In [ ]:
def check_relevance(prompt, document_text,text_splitter, threshold=0.7):
  vector_store = FAISS.from_documents(document_text, text_splitter.embeddings)
  relevant_result = vector_store.similarity_search_with_score(prompt, k=1)
  if relevant_result and relevant_result[0][1] >= threshold:
    return True  # Prompt is relevant
  else:
    return False

In [ ]:
text_splitter = SemanticChunker(OpenAIEmbeddings())

In [ ]:
path="/content/sample_data/Docs"
documents=load_document(path)

In [ ]:
text_split=text_splitter.split_documents(documents)

In [ ]:
parser=StrOutputParser()
model=ChatOpenAI()

In [ ]:
prompt_text="Why should machines have to learn, answer me in one line"
prompt=generate_response(prompt_text)
result=parser.parse(prompt)
print(result)

content='To adapt and improve their functionality and capabilities over time.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 19, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BKMmCFZIZhPpmMvLBfzCZ5u1mxxPL', 'finish_reason': 'stop', 'logprobs': None} id='run-e2a0637d-0735-4636-a958-ca844049b0a8-0' usage_metadata={'input_tokens': 19, 'output_tokens': 12, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
